In [1]:
if not os.path.isfile("estrutura_ud.py"):
    ! wget https://raw.githubusercontent.com/alvelvis/ACDC-UD/master/estrutura_ud.py
import estrutura_ud
import os

In [17]:
file = "../Bosque/bosque-workbench.conllu"
#file = "../Petroles_v2/Petroles_2.conllu"
corpus = estrutura_ud.Corpus()
corpus.load(file)

for i in range(2):
    for sentence in corpus.sentences.values():
        if sentence.sent_id not in [
            "1-20140905-TESEDSC_0_resumo-11",
            "32-20141110-MONOGRAFIA_0_resumo-19",
            "36-20140327-TESEMSC_0_resumo-6",
            "51-20140924-MONOGRAFIA_0_resumo-1",
            "6-20140908-MONOGRAFIA_0_intro-53",
            '105-20151215-MONOGRAFIA_0_resumo-8',
            "32-20141110-MONOGRAFIA_0_resumo-13",
            "20-20140904-TESEDSC_0-59",
            "20-20140904-TESEDSC_0-66",
            "19-20140916-TESEDSC_0_resumo-9",
            "CF47-3",
            "CF51-2",
            "CF68-2",
            "CF122-3",
            "CF124-3",
            "CF151-31",
            "CF423-5",
            "CF476-4",
            "CF479-1",
            "CF602-4",
            "CP161-4",
            "CF288-7",
            "CP782-2"
        ]:
            tokens = sentence.tokens
            for token in tokens:

                if not '-' in token.id and token.deprel not in ["appos:parataxis", "compound", "flat:foreign", "orphan", "flat", "fixed", "orphan"]:
                    has_cc = False
                    has_colon = False
                    has_comma = False
                    has_parenthesis = False
                    has_pronrel = False
                    for _token in tokens:
                        #if _token.word in [",", "-", "–", ";", "--", "...", ":"] and _token.dephead == token.head_token.id and token.deprel == "appos" and token != _token:
                            #_token.dephead = token.id
                        if _token.dephead == token.id:
                            if _token.deprel == "cc":
                                has_cc = True
                            if _token.word in [",", "-", "–", ";", "--", "..."]:
                                has_comma = True
                            if _token.word in ["(", "["]:
                                has_parenthesis = True
                            if "PronType=Rel" in _token.feats:
                                has_pronrel = True
                        if _token.dephead == token.id:
                            if _token.word == ":":
                                has_colon = True

                    if token.word in ["um", "uma", "uns", "umas"] and token.next_token.word == "de":
                        if sentence.sent_id not in ["CP659-1"]:
                            token.upos = "PRON"
                            token.lemma = "um"
                            token.feats = "Gender={}|Number={}|PronType=Ind".format('Fem' if 'a' in token.word.lower() else "Masc", "Plur" if 's' in token.word.lower() else "Sing")

                    if token.upos in ["PRON", "NOUN"] and token.head_token.upos == "NOUN" and int(token.id) > int(token.head_token.id):
                        if not any([has_comma, has_cc, has_colon, has_parenthesis, has_pronrel]) and token.deprel not in ["nsubj", "acl"]:
                            token.deprel = "nmod"

                    if token.upos == "NUM" and token.head_token.upos == "NOUN" and int(token.id) > int(token.head_token.id):
                        if not any([has_comma, has_cc, has_colon, has_parenthesis]):
                            token.deprel = "nummod"

                    if token.upos == "PROPN" and token.head_token.upos in ["NOUN"] and int(token.id) > int(token.head_token.id):
                        if sentence.sent_id not in [
                            "CF473-3",
                            "CF504-1",
                            "CF536-7",
                            "CF666-5",
                            "CF931-3",
                            "CP233-6",
                            "CP791-7"
                            ]:
                            if not any([has_colon, has_comma, has_parenthesis, has_cc, has_pronrel]) and token.deprel != "nsubj":
                                if token.previous_token.word in [",", ":", "--", ";"] and token.deprel in ["appos", "conj"]:
                                    if int(token.head_token.id) > int(token.previous_token.id):
                                        token.previous_token.dephead = token.head_token.id
                                    else:
                                        token.previous_token.dephead = token.id
                                elif token.previous_token.previous_token.word in [",", ":", "--", ";"] and token.deprel in ["appos", "conj"] and int(token.previous_token.previous_token.dephead) < int(token.previous_token.previous_token.id):
                                    if int(token.head_token.id) > int(token.previous_token.previous_token.id):
                                        token.previous_token.previous_token.dephead = token.head_token.id
                                    else:
                                        token.previous_token.previous_token.dephead = token.id
                                elif token.previous_token.id != "1" and token.previous_token.previous_token.previous_token.word in [",", ":", "--", ";"] and token.deprel in ["appos", "conj"] and int(token.previous_token.previous_token.previous_token.dephead) < int(token.previous_token.previous_token.previous_token.id):
                                    if int(token.head_token.id) > int(token.previous_token.previous_token.previous_token.id):
                                        token.previous_token.previous_token.previous_token.dephead = token.head_token.id
                                    else:
                                        token.previous_token.previous_token.previous_token.dephead = token.id
                                else:
                                    token.deprel = "nmod"

                    if token.upos == "PROPN" and token.head_token.upos == "PROPN" and int(token.id) > int(token.head_token.id):
                        if sentence.sent_id not in [
                            "CF178-2",
                            "CF183-1",
                            "CF248-1",
                            "CF288-5",
                            "CF298-4",
                            "CF307-3",
                            "CF308-4",
                            "CF329-3",
                            "CF380-3",
                            "CF384-5",
                            "CF409-4"
                            ]:
                            if not any([has_cc, has_comma, has_parenthesis]):
                                token.deprel = "flat:name"
                                if token.head_token == "flat:name":
                                    token.dephead = token.head_token.dephead
                                #for _token in tokens:
                                    #if _token.dephead == token.id:
                                        #if _token.upos in ["ADP", "DET", "PUNCT", "CCONJ"] and int(_token.id) > int(token.head_token.id): #or _token.word in ["--"]:
                                        #_token.dephead = token.dephead
                                for _token in tokens:
                                    if (_token.dephead == token.head_token.id or _token.dephead == token.id) and int(_token.id) > int(token.head_token.id) and int(_token.id) < int(token.id):
                                        if _token.word not in ["(", ")", "«", "»"]:
                                            _token.deprel = "flat:name"
                                        else:
                                            break

                    if token.head_token.deprel == "flat:name" and token.word not in ["(", ")"]:# and token.upos in ["ADP", "DET", "PUNCT", "CCONJ"]:
                        token.dephead = token.head_token.dephead# if token.head_token.head_token.deprel != "flat:name" else token.head_token.head_token.dephead
                        #if not any([has_cc, has_comma]):
                        #token.deprel = "flat:name"
                    

corpus.save("teste.conllu")
os.system("meld --diff {} teste.conllu".format(file))
os.remove("teste.conllu")

build: 10.978663682937622